<a href="https://colab.research.google.com/github/stevengregori92/Pytorch-Intro/blob/main/Callbacks_and_more_ML_Engineer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Jcopdl Libraries

In [ ]:
!pip install luwiji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 MB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.0 MB/s eta 0:00:00
  Created wheel for luwiji: filename=luwiji-1.2.1-py2.py3-none-any.whl size=40366830 sha256=cb6dc52ff58622ae01c5821a0a88ad706f99b702e040ab3d3db65e8559c3582d
  Stored in directory: /root/.cache/pip/wheels/ba/90/9e/943ac5067e9cdc185d8f029a1d13278acd52839118acc06c9a
  Created wheel for jcopml: filename=jcopml-1.2.2-py2.py3-none-any.whl size=32178 sha256=1c8b74ab812bb6d1c84384a15e9ab53a06d0f323391caef88870c5300069abdc
  Stored in directory: /root/.cache/pip/wheels/5e/06/58/749e5f1baf2932285758798a92793d4347888826160b025808
Successfully built luwiji jcopml


In [ ]:
!pip install "jcopdl<2.0"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for jcopdl: filename=jcopdl-1.1.10-py2.py3-none-any.whl size=17913 sha256=a69c7a13b589d82d25fc1b70f58b5c44247a95674a4ce3ac9c925299571714e5
  Stored in directory: /root/.cache/pip/wheels/41/95/30/86345d2446be19c7d97dee789a2597bee81cfbb7b24a847f7c
Successfully built jcopdl


In [ ]:
import torch
from torch import nn, optim
from jcopdl.callback import Callback, set_config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

#Dataset & Dataloader

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
bs=128

transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor()
])

train_set = datasets.ImageFolder('drive/MyDrive/train/', transform=transform)
trainloader = DataLoader(train_set, batch_size=bs, shuffle = True, num_workers=2)

test_set = datasets. ImageFolder('drive/MyDrive/test/', transform=transform)
testloader = DataLoader(test_set, batch_size=bs, shuffle = True, num_workers=2)

In [ ]:
images, labels = next(iter(trainloader))
images.shape

torch.Size([128, 1, 28, 28])

In [ ]:
label2cat = train_set.classes
label2cat

['Ankle boot',
 'Bag',
 'Coat',
 'Dress',
 'Pullover',
 'Sandal',
 'Shirt',
 'Sneaker',
 'T-shirt or Top',
 'Trouser']

#Arsitektur & Config

In [ ]:
from jcopdl.layers import linear_block

In [ ]:
class FashionClassifier(nn.Module):
  def __init__(self, input_size, output_size, n1, n2, dropout=0.2):
    super().__init__()
    self.fc = nn.Sequential(
        nn.Flatten(),
        linear_block(input_size, n1, dropout=dropout),
        linear_block(n1, n2, dropout=dropout),
        linear_block(n2, output_size, activation='lsoftmax')
    )

  def forward(self, x):
    return self.fc(x)

In [ ]:
config = set_config({
    'input_size': 784,
    'output_size': len(train_set.classes),
    'n1': 256,
    'n2': 64,
    'dropout': 0.2  
})

#Training Preparation -> MCOC

In [ ]:
model = FashionClassifier(config.input_size, config.output_size, config.n1, config.n2, config.dropout).to(device)
criterion = nn.NLLLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)
callback = Callback(model, config, outdir='model')

#DL Training

In [ ]:
from tqdm.auto import tqdm

def loop_fn(mode, dataset, dataloader, model, criterion, optimizer, device):
    if mode == 'train':
        model.train()
    elif mode == 'test':
        model.eval()
    cost = correct = 0
    for feature, target in tqdm(dataloader, desc=mode.title()):
        feature, target = feature.to(device), target.to(device) # corrected line
        output = model(feature)
        loss = criterion(output, target)

        if mode == 'train':
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        cost += loss.item() * feature.shape[0]
        correct += (output.argmax(1) == target).sum().item()
        
    cost = cost/len(dataset)
    acc = correct/len(dataset)
    return cost, acc

In [ ]:
while True:
  train_cost, train_score = loop_fn('train', train_set, trainloader, model, criterion, optimizer, device)
  with torch.no_grad():
      train_cost, train_score = loop_fn('test', test_set, testloader, model, criterion, optimizer, device)

  #Logging
  callback.log(train_cost, test_cost, train_score, test_score)

  #Checkpoint
  callback.save_checkpoint()

  #Runtime Plotting
  callback.cost_runtime_plotting()
  callback.score_runtime_plotting()

  #Early Stopping
  if callback.early_stopping(model, monitor='test_score'):
    callback.plot_cost()
    callback.plot_score()
    break

Train:   0%|          | 0/313 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

#Load Best Model

In [ ]:
weights = torch.load('model/weights_best.pth', map_location='cpu')
config = torch.load('model/configs.pth', map_location='cpu')

model = FashionClassifier(config.input_size, config.output_size, config.n1, config.n2, config.dropout)
mode.load_state_dict(weights)
model = model.to(weights)

#DL Predict

In [ ]:
feature, target = next(iter(testloader))
feature, target = feature.to(device), target.to(device)

In [ ]:
with torch.no_grad():
  model.eval()
  output.model(feature)
  preds = output.argmax(1)
[label2cat[pred] for pred in preds]

#DL Visualize

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, axes = plt.subplots(6, 6, figsize=(24,24))
for img, label, pred, ax in zip(feature, target, preds, axes.flatten()):
  ax.imshow(img.view(28,28).cpu(), cmap='gray')
  font={'color': 'r'} if label != preds else {'color': 'g'}
  label, pred = label2cat[label.item()], label2cat[pred.item()]
  ax.set_title(f'Label: {label} | Pred: {pred}', fontdict=font);
  ax.show('off');